In [113]:
#This version of the evolutionary algorithm seeks to implement
#random migration and tune hyperparameters

In [1]:
import numpy as np
import time
from statistics import mean

In [2]:
def selection(pop, scores, k=3):
    selection_ix = np.random.randint(len(pop))
    for ix in np.random.randint(0, len(pop), k-1):
        if scores[ix] > scores[selection_ix]:
            selection_ix = ix
    return pop[selection_ix]

def crossover(p1, p2, r_cross):
    c1, c2 = p1.copy(), p2.copy()
    if np.random.rand() < r_cross:
        pt = np.random.randint(1, len(p1)-1)
        c1 = p1[:pt] + p2[pt:]
        c2 = p2[:pt] + p1[pt:]
    return [c1, c2]

def mutation(bitstring, r_mut):
    for i in range(len(bitstring)):
        if np.random.rand() < r_mut:
            bitstring[i] = np.random.randint(0, 4)#1 - bitstring[i]
    return bitstring

In [3]:
#mutation([1] * 10 + [0] * 10, 1/4)

In [4]:
def update_consumergoods(state, toaster_percent):
    match state:
        case 7:
            toaster_percent -= 0.15 #WAR ECO     
        case 30:
            toaster_percent -= 0.05 #construction company
        case 34:
            toaster_percent -= 0.03 #stability bonus
    return round(toaster_percent, 2)

def update_civs(state, civs):
    match state:
        case 8:
            civs += 2
        case 20:
            civs += 2
        case 22:
            civs += 2
        case 25:
            civs += 2
        case 34:
            civs += 13 #annexation of baltics        
    return civs

def update_mils(state, mils):
    match state:
        case 34:
            mils += 8    
    return mils


def update_constructionspeed(state, civ_output_modifier, scenario):
    match scenario:
        case 'one year':
            match state:
                case 0:
                    civ_output_modifier += 0.1 #export focus
                case 5:
                    civ_output_modifier += 0.1
                case 12: #same as half year because we start in 36
                    civ_output_modifier += 0.1 
                case 31:
                    civ_output_modifier += 0.1
                case 52:
                    civ_output_modifier += 0.1
        case 'half year':
            match state:
                case 0:
                    civ_output_modifier += 0.1 #export focus
                case 5:
                    civ_output_modifier += 0.1
                case 12: 
                    civ_output_modifier += 0.1 
                case 33:
                    civ_output_modifier += 0.1
                case 54:
                    civ_output_modifier += 0.1
        case 'current':
            match state:
                case 0:
                    civ_output_modifier += 0.1 #export focus
                case 5:
                    civ_output_modifier += 0.1
                case 16: 
                    civ_output_modifier += 0.1 
                case 36:
                    civ_output_modifier += 0.1
                case 57:
                    civ_output_modifier += 0.1
            
    return round(civ_output_modifier, 2)

def update_miloutput(state, mil_output_modifier, scenario):
    match state: #These are the bonuses related to stability
        case 0:
            mil_output_modifier -= 0.2180
        case 2: 
            mil_output_modifier += 0.05
        case 4: 
            mil_output_modifier += 0.05
        case 8:
            mil_output_modifier += 0.10
        case 9: #At this point we are above 50% stability
            mil_output_modifier += 0.04
        case 10: #popular figurehead
            mil_output_modifier += 0.06 
        case 24: #kill trotsky + new NKVD head
            mil_output_modifier += 0.04
    
    match scenario:
        case 'one year':
            match state:
                case 9:
                    mil_output_modifier += 0.15
                case 14: 
                    mil_output_modifier += 0.15
                case 31:
                    mil_output_modifier += 0.15
                case 52:
                    mil_output_modifier += 0.15
        case 'half year':
            match state:
                case 9:
                    mil_output_modifier += 0.15
                case 14: 
                    mil_output_modifier += 0.15
                case 33:
                    mil_output_modifier += 0.15
                case 54:
                    mil_output_modifier += 0.15
        case 'current':
            match state:
                case 9:
                    mil_output_modifier += 0.15
                case 16: 
                    mil_output_modifier += 0.15
                case 36:
                    mil_output_modifier += 0.15
                case 57:
                    mil_output_modifier += 0.15

    return round(mil_output_modifier, 3)

def update_prodefficiency(state, production_efficiency_cap, scenario):
    match state: #These are related to the 'defense industry' buff in the focus tree
        case 30:
            production_efficiency_cap += 0.05
        case 40:
            production_efficiency_cap += 0.05
    
    match scenario:
        case 'one year':
            match state:
                case 4:
                    production_efficiency_cap += 0.1
                case 13:
                    production_efficiency_cap += 0.1 
                case 26: #construction company american experts
                    production_efficiency_cap += 0.1 
                case 31:
                    production_efficiency_cap += 0.1
                case 52:
                    production_efficiency_cap += 0.1
        case 'half year':
            match state:
                case 4:
                    production_efficiency_cap += 0.1
                case 13: 
                    production_efficiency_cap += 0.1 
                case 26: #construction company american experts
                    production_efficiency_cap += 0.1 
                case 33:
                    production_efficiency_cap += 0.1
                case 54:
                    production_efficiency_cap += 0.1
        case 'current':
            match state:
                case 4:
                    production_efficiency_cap += 0.1
                case 15: 
                    production_efficiency_cap += 0.1 
                case 26: #construction company american experts
                    production_efficiency_cap += 0.1 
                case 36:
                    production_efficiency_cap += 0.1
                case 57:
                    production_efficiency_cap += 0.1

    return round(production_efficiency_cap, 2)

def update_civcost(state, civ_construction_modifier):
    match state:
        case 0:
            civ_construction_modifier += 0.05 #five year plan
        case 7:
            civ_construction_modifier += 0.3
        case 12: 
            civ_construction_modifier += 0.1 #captain of industry
        case 21:
            civ_construction_modifier += 0.05 #construction company
        case 30:
            civ_construction_modifier += 0.05 #construction company upgrade
    return round(civ_construction_modifier, 2)

def update_milcost(state, mil_construction_modifier):
    match state:
        case 7:
            mil_construction_modifier += 0.5
        case 21:
            mil_construction_modifier += 0.05 #construction company
    return round(mil_construction_modifier, 2)

def update_civ_to_milcost(state, civ_to_mil_construction_modifier):
    match state:
        case 7:
            civ_to_mil_construction_modifier += 0.5
    return round(civ_to_mil_construction_modifier, 2)

def update_mil_to_civcost(state, mil_to_civ_construction_modifier):
    match state:
        case 7:
            mil_to_civ_construction_modifier += 0.5
    return round(mil_to_civ_construction_modifier, 2)

def prod_eff_calculations(state, prod_eff, prod_eff_cap, factory_state_array, new_mils):
    factory_count_list = []
    efficiency_list = []
    sum_of_efficiency = 0
    prod_eff = 0
    factory_state_array[state][0] = new_mils
    factory_state_array[state][1] = 0.1
    
    if state == 0:
        factory_state_array[state][1] = 0.5
    
    for i in range(state):
        factory_count_list.append(factory_state_array[i][0])
        efficiency_list.append(factory_state_array[i][1])   
        growth = 0.001 * (prod_eff_cap ** 2 / factory_state_array[i][1]) * 17.5
        factory_state_array[i][1] += growth
        factory_state_array[i][1] = min(factory_state_array[i][1], 1)
        
    num_factories = sum(factory_count_list)
    
    for j in range(len(factory_count_list)):
        sum_of_efficiency += factory_count_list[j] * efficiency_list[j]
    
    try:
        return min(sum_of_efficiency / num_factories, 1)
    except:
        return 0.5 #This should only happen at the start, then eff = 0.5

In [5]:
#Five max_targets: 'rush', 'scale', 'scale_total', 'scale_civs' and 'mils'

def production(actions, max_target, scenario):
    
    st = time.time()
    mils = 32
    mils_0 = 32 #keep track of the number of starting factories
    new_mils = 32 #initialize production efficiency
    civs = 45 + 8 #(a baseline number of factories from trade)
    civs_0 = 45 #idem
    total = mils + civs
    total_0 = 32 + 45 #idem
    consumergoods_percentage = 0.40
    consumergoods = np.floor(total * consumergoods_percentage)
    available_civs = np.maximum(int(civs - consumergoods), 0)
    
    #Civ output modifiers are additive (-30% + 15% = -15%), multiplied by state infrastructure
    civ_output = 5 
    civ_output_modifier = 1 #Construction speed
    civ_construction_modifier = 0.7
    mil_construction_modifier = 0.7
    civ_to_mil_construction_modifier = 0.7
    mil_to_civ_construction_modifier = 0.7
    
    mil_output = 4.5
    mil_output_modifier = 1
    prod_eff = 0.5
    production_efficiency_cap = 0.55
    
    state = 0
    state_size_days = 35 #how many days per state
    ic_produced = 0
    ic_total = 0    
    civ_purchase_pool = 0 
    mil_purchase_pool = 0
    civ_to_mil_pool = 0
    mil_to_civ_pool = 0
    infrastructure_modifier = 1.6
    
    factory_state_array = np.empty((57), dtype=object)

    for i in range(len(factory_state_array)):
        factory_state_array[i] = np.zeros(2)
    
    for i in range(len(actions)):
        
        civs = (update_civs(i, civs))
        mils = (update_mils(i, mils))
        consumergoods_percentage = (update_consumergoods(i, consumergoods_percentage))
        available_civs = np.maximum(int(civs - consumergoods), 0)
        civ_output_modifier = (update_constructionspeed(i, civ_output_modifier, scenario))
        mil_output_modifier = (update_miloutput(i, mil_output_modifier, scenario))
        civ_construction_modifier = (update_civcost(i, civ_construction_modifier))
        mil_construction_modifier = (update_milcost(i, mil_construction_modifier))
        civ_to_mil_construction_modifier = (update_civ_to_milcost(i, civ_to_mil_construction_modifier))
        mil_to_civ_construction_modifier = (update_mil_to_civcost(i, mil_to_civ_construction_modifier))
        production_efficiency_cap = (update_prodefficiency(i, production_efficiency_cap, scenario))
        
        ic_produced = mil_output * mils * state_size_days * mil_output_modifier * prod_eff
        prod_eff = prod_eff_calculations(i, prod_eff, production_efficiency_cap, factory_state_array, new_mils)

        if actions[i] == 0: #build mils     
            mil_purchase_pool += (available_civs * civ_output * state_size_days
                                 * infrastructure_modifier * civ_output_modifier
                                 * mil_construction_modifier)
            existing_mils = mils #Used exclusively for production efficiency calculations
            new_mils = np.floor_divide(mil_purchase_pool, 7200) #for production efficiency
            mils += np.floor_divide(mil_purchase_pool, 7200) 
            total += np.floor_divide(mil_purchase_pool, 7200) 
            consumergoods = np.ceil(total * consumergoods_percentage)
            mil_purchase_pool = np.mod(mil_purchase_pool, 7200)
            available_civs = np.maximum(int(civs - consumergoods), 0)
                     
            state += 1
            ic_total += ic_produced
            
        if actions[i] == 1: #build civs         
            civ_purchase_pool += (available_civs * civ_output * state_size_days
                                 * infrastructure_modifier * civ_output_modifier
                                 * civ_construction_modifier)

            new_mils = 0 #Account for annexations here
            civs += np.floor_divide(civ_purchase_pool, 10800) 
            total += np.floor_divide(civ_purchase_pool, 10800) 
            consumergoods = np.ceil(total * consumergoods_percentage)
            civ_purchase_pool = np.mod(civ_purchase_pool, 10800)
            available_civs = np.maximum(int(civs - consumergoods), 0)
            
            state += 1
            ic_total += ic_produced

        if actions[i] == 2: #Convert civs to mils
            civ_to_mil_pool += (available_civs * civ_output * state_size_days
                                 * infrastructure_modifier * civ_output_modifier
                                 * civ_to_mil_construction_modifier)
            new_mils = np.floor_divide(civ_to_mil_pool, 4000)
            civs -= np.floor_divide(civ_to_mil_pool, 4000)
            mils += np.floor_divide(civ_to_mil_pool, 4000)
            civ_to_mil_pool = np.mod(civ_to_mil_pool, 4000)
            consumergoods = np.ceil(total * consumergoods_percentage)
            available_civs = np.maximum(int(civs - consumergoods), 0)
            ic_total += ic_produced
        
        if actions[i] == 3: #Convert mils to civs
            mil_to_civ_pool += (available_civs * civ_output * state_size_days
                                 * infrastructure_modifier * civ_output_modifier
                                 * mil_to_civ_construction_modifier)
            if mils > 0:
                new_mils = 0
                civs += np.floor_divide(mil_to_civ_pool, 9000)
                mils -= np.floor_divide(mil_to_civ_pool, 9000)
                
                if mils < 0:
                    too_many_removed = abs(mils)
                    civs -= too_many_removed
                    mils = 0
                    mil_to_civ_pool = 0
                
                else:
                    mil_to_civ_pool = np.mod(mil_to_civ_pool, 9000)
                consumergoods = np.ceil(total * consumergoods_percentage)
                available_civs = np.maximum(int(civs - consumergoods), 0)
            ic_total += ic_produced
            
    et = time.time()
    elapsed_time = et - st
    #print("Time taken: " + str(elapsed_time) + " seconds")
            
    match max_target:
        case 'rush':
            return ((ic_total, mils, civs))
        case 'scale':
            scaled_score = ic_total * np.log2((mils/mils_0))
            return ((scaled_score, mils, civs, ic_total))
        case 'scale_total':
            scaled_score = ic_total * np.log2((mils/mils_0)) * (total/total_0)
            if scaled_score == None:
                scaled_score = 0
            return ((scaled_score, mils, civs, ic_total))
        case 'scale_civs':
            scaled_score = ic_total * np.log2((mils/mils_0)) * np.log2((civs/civs_0))
            if scaled_score == None:
                scaled_score = 0
            return ((scaled_score, mils, civs, ic_total))
        case 'mils':
            return ((mils, civs, ic_total))

In [6]:
def genetic_algorithm(n_bits, n_pop, n_iter, r_cross, max_target, scenario, mutation_rate, elitism, migration):
    
    st = time.time()
    pop = [np.random.randint(0, 4, n_bits).tolist() for _ in range(n_pop)]
    best, best_eval = 0, production(pop[0], max_target, scenario)
    gen_since_last_improvement = 0
    
    for gen in range(n_iter):
        #print('generation ' + str(gen) + ' population: ' + str(pop))
        if gen_since_last_improvement < 10:
            gen_since_last_improvement += 1
            scores = [production(c, max_target, scenario) for c in pop]
            for i in range(n_pop):
                if scores[i] > best_eval:
                    best, best_eval = pop[i], scores[i]
                    gen_since_last_improvement = 0
                    #print("Generation " + str(gen) + ": new best solution found!")
                    #print(best, best_eval)
                
        
            selected = [selection(pop, scores) for _ in range(n_pop)]
            children = []
            for i in range(0, n_pop, 2):
                p1, p2 = selected[i], selected[i+1]
                c = crossover(p1, p2, r_cross)
                for child in c:
                    mutated = mutation(child, mutation_rate)
                    children.append(mutated)
                    
            #Add random new samples
            if gen % 10 == 0 and gen > 0 and migration:
                del children[-10:]
                migration = [np.random.randint(0, 4, n_bits).tolist() for _ in range(10)]
                for i in migration:
                    children.append(i)
            
            if gen > 1 and elitism:
                del children[-1] #Delete the last element and replace with best solution
                children.append(best)

            pop = children
        else:
            break

    et = time.time()
    elapsed_time = et - st
    print("Time taken: " + str(elapsed_time) + " seconds")
    
    return [best, best_eval]

In [7]:
#genetic_algorithm(57,100, 600, 0.8, 'scale_total', 'half year', 1 / 57, True, False)

In [8]:
#Which parameters should I be tuning?

def hyperparameter_tuning():
    n_pop = [100, 200, 500]
    r_cross = [0.80, 0.90, 1]
    mutation_rate = [0.001, 0.01, 1 / 57, 0.1]
    all_performances_average = []
    
    for i in n_pop:
        for j in r_cross:
            for k in mutation_rate:
                performance_list = [] #Reset this list per different setting
                for l in range(0, 10):
                    result = genetic_algorithm(57, i, 600, j, 'scale_total', 'half year', k, False, True)
                    performance_list.append(round(result[1][0], 2))
                print('N_pop: {}, R_cross {}, Mutation_rate {}: '.format(i, j, k) + str(mean(performance_list)))
                all_performances_average.append((mean(performance_list), i, j, k))
    return sorted(all_performances_average, reverse = True)

In [9]:
tuning = hyperparameter_tuning()
tuning

Time taken: 35.5053551197052 seconds
Time taken: 49.48041009902954 seconds
Time taken: 15.243200540542603 seconds
Time taken: 36.45685648918152 seconds
Time taken: 34.135568141937256 seconds
Time taken: 38.13075017929077 seconds
Time taken: 51.686429262161255 seconds
Time taken: 26.003784656524658 seconds
Time taken: 26.27325987815857 seconds
Time taken: 38.111576080322266 seconds
N_pop: 100, R_cross 0.8, Mutation_rate 0.001: 5416481.652
Time taken: 56.71615386009216 seconds
Time taken: 48.20537209510803 seconds
Time taken: 47.760666370391846 seconds
Time taken: 61.51595115661621 seconds
Time taken: 45.80015850067139 seconds
Time taken: 40.91883111000061 seconds
Time taken: 55.31175088882446 seconds
Time taken: 52.079594135284424 seconds
Time taken: 52.626227617263794 seconds
Time taken: 36.25334596633911 seconds
N_pop: 100, R_cross 0.8, Mutation_rate 0.01: 6047130.399
Time taken: 39.40996551513672 seconds
Time taken: 33.63505721092224 seconds
Time taken: 39.7501060962677 seconds
Time 

Time taken: 70.78246736526489 seconds
Time taken: 60.03685975074768 seconds
Time taken: 67.4589421749115 seconds
Time taken: 69.85697555541992 seconds
N_pop: 200, R_cross 0.9, Mutation_rate 0.017543859649122806: 6128092.166
Time taken: 23.35840916633606 seconds
Time taken: 22.254327535629272 seconds
Time taken: 24.10670042037964 seconds
Time taken: 32.022027254104614 seconds
Time taken: 33.599522829055786 seconds
Time taken: 33.816983222961426 seconds
Time taken: 48.5634970664978 seconds
Time taken: 23.378908395767212 seconds
Time taken: 24.956568241119385 seconds
Time taken: 39.009413719177246 seconds
N_pop: 200, R_cross 0.9, Mutation_rate 0.1: 4760286.584
Time taken: 49.83085823059082 seconds
Time taken: 62.83990526199341 seconds
Time taken: 43.49266839027405 seconds
Time taken: 66.77181673049927 seconds
Time taken: 66.14410257339478 seconds
Time taken: 77.69982695579529 seconds
Time taken: 67.9331784248352 seconds
Time taken: 66.50822591781616 seconds
Time taken: 57.64480423927307 s

[(6148971.013, 500, 1, 0.01),
 (6148087.92, 500, 1, 0.017543859649122806),
 (6146698.551, 500, 0.9, 0.017543859649122806),
 (6143838.6049999995, 500, 0.8, 0.017543859649122806),
 (6140194.619, 500, 0.8, 0.01),
 (6129599.678, 200, 1, 0.017543859649122806),
 (6128092.166, 200, 0.9, 0.017543859649122806),
 (6115455.506, 500, 0.9, 0.01),
 (6106649.362, 200, 0.8, 0.017543859649122806),
 (6097232.341, 200, 0.8, 0.01),
 (6095438.024, 500, 0.9, 0.001),
 (6068637.379, 200, 0.9, 0.01),
 (6066762.222, 200, 1, 0.01),
 (6050373.821, 100, 1, 0.017543859649122806),
 (6050246.996, 500, 1, 0.001),
 (6047130.399, 100, 0.8, 0.01),
 (6046591.481, 100, 0.8, 0.017543859649122806),
 (6041961.114, 100, 0.9, 0.017543859649122806),
 (6040103.001, 500, 0.8, 0.001),
 (5997696.447, 100, 0.9, 0.01),
 (5991088.158, 100, 1, 0.01),
 (5892165.871, 200, 1, 0.001),
 (5852971.688, 200, 0.9, 0.001),
 (5833457.4969999995, 200, 0.8, 0.001),
 (5713666.824, 100, 1, 0.001),
 (5667653.621, 100, 0.9, 0.001),
 (5416481.652, 100, 0

In [131]:
#BEST PERFORMING SETTINGS

#genetic_algorithm(57,500, 600, 0.8, 'scale_civs', 'half year', 1 / 57, True, False)
#about 150 seconds with a population of 500

In [180]:
#genetic_algorithm(57,200, 600, 0.9, 'scale_civs', 'half year', 1 / 57)
#about 60 seconds with a population of 300

In [133]:
#genetic_algorithm(57,100, 600, 0.9, 'scale_civs', 'half year')
#about 35 seconds with a population of 1

In [9]:
tuning_data_elitism = [(6151354.4969999995, 500, 0.8, 0.017543859649122806),
 (6141190.765, 500, 0.8, 0.01),
 (6139304.692, 500, 0.9, 0.017543859649122806),
 (6138229.692, 500, 1, 0.01),
 (6137890.265, 500, 1, 0.017543859649122806),
 (6131516.4059999995, 500, 0.9, 0.01),
 (6126966.847, 200, 1, 0.017543859649122806),
 (6124779.81, 200, 0.9, 0.017543859649122806),
 (6095898.870999999, 200, 0.9, 0.01),
 (6088720.391, 200, 0.8, 0.017543859649122806),
 (6058727.3100000005, 500, 0.8, 0.001),
 (6049658.739, 500, 1, 0.001),
 (6048796.828, 100, 1, 0.017543859649122806),
 (6046341.458, 100, 0.8, 0.017543859649122806),
 (6045110.3889999995, 200, 0.8, 0.01),
 (6037965.603, 100, 0.8, 0.01),
 (6037130.596, 100, 0.9, 0.017543859649122806),
 (6034854.692, 200, 1, 0.01),
 (6030770.13, 100, 1, 0.01),
 (6021191.24, 100, 0.9, 0.01),
 (5987066.933, 500, 0.9, 0.001),
 (5947535.682, 200, 0.8, 0.001),
 (5891037.638, 200, 1, 0.001),
 (5854684.155, 200, 0.9, 0.001),
 (5700835.669, 100, 0.9, 0.001),
 (5605893.064, 100, 1, 0.001),
 (5561471.321, 100, 0.8, 0.001),
 (5170051.133, 500, 1, 0.1),
 (5061097.736, 500, 0.9, 0.1),
 (5034034.569, 200, 0.9, 0.1),
 (4979460.757, 100, 0.8, 0.1),
 (4969699.640000001, 500, 0.8, 0.1),
 (4955521.856, 200, 0.8, 0.1),
 (4935814.575, 100, 0.9, 0.1),
 (4921377.552, 200, 1, 0.1),
 (4915185.546, 100, 1, 0.1)]

for i in range(len(tuning_data_elitism)):
    tuning_data_elitism[i] = (tuning_data_elitism[i][0],tuning_data_elitism[i][1], 
                              tuning_data_elitism[i][2], tuning_data_elitism[i][3], 'elit_no_migr')
    
print(tuning_data_elitism)

[(6151354.4969999995, 500, 0.8, 0.017543859649122806, 'elit_no_migr'), (6141190.765, 500, 0.8, 0.01, 'elit_no_migr'), (6139304.692, 500, 0.9, 0.017543859649122806, 'elit_no_migr'), (6138229.692, 500, 1, 0.01, 'elit_no_migr'), (6137890.265, 500, 1, 0.017543859649122806, 'elit_no_migr'), (6131516.4059999995, 500, 0.9, 0.01, 'elit_no_migr'), (6126966.847, 200, 1, 0.017543859649122806, 'elit_no_migr'), (6124779.81, 200, 0.9, 0.017543859649122806, 'elit_no_migr'), (6095898.870999999, 200, 0.9, 0.01, 'elit_no_migr'), (6088720.391, 200, 0.8, 0.017543859649122806, 'elit_no_migr'), (6058727.3100000005, 500, 0.8, 0.001, 'elit_no_migr'), (6049658.739, 500, 1, 0.001, 'elit_no_migr'), (6048796.828, 100, 1, 0.017543859649122806, 'elit_no_migr'), (6046341.458, 100, 0.8, 0.017543859649122806, 'elit_no_migr'), (6045110.3889999995, 200, 0.8, 0.01, 'elit_no_migr'), (6037965.603, 100, 0.8, 0.01, 'elit_no_migr'), (6037130.596, 100, 0.9, 0.017543859649122806, 'elit_no_migr'), (6034854.692, 200, 1, 0.01, 'el

In [10]:
tuning_data_no_elitism = [(6148712.234999999, 500, 0.9, 0.01),
 (6146679.5709999995, 500, 0.8, 0.017543859649122806),
 (6146156.528, 500, 0.9, 0.017543859649122806),
 (6146140.344, 500, 1, 0.01),
 (6137345.623, 500, 1, 0.017543859649122806),
 (6131581.3319999995, 500, 0.8, 0.01),
 (6123178.155, 200, 0.8, 0.017543859649122806),
 (6104209.779, 200, 1, 0.01),
 (6099314.348, 200, 0.9, 0.017543859649122806),
 (6087498.12, 200, 1, 0.017543859649122806),
 (6073420.446, 100, 0.9, 0.017543859649122806),
 (6067200.394, 100, 0.8, 0.017543859649122806),
 (6060090.344, 200, 0.8, 0.01),
 (6056372.988, 100, 1, 0.017543859649122806),
 (6031386.987, 200, 0.9, 0.01),
 (6031222.557, 500, 1, 0.001),
 (6024036.263, 100, 1, 0.01),
 (6012174.932, 500, 0.9, 0.001),
 (6009463.068, 100, 0.8, 0.01),
 (6001872.948, 500, 0.8, 0.001),
 (5983476.973999999, 100, 0.9, 0.01),
 (5876831.446, 200, 0.8, 0.001),
 (5831667.22, 200, 1, 0.001),
 (5826168.898, 200, 0.9, 0.001),
 (5538917.24, 100, 1, 0.001),
 (5456670.258, 100, 0.8, 0.001),
 (5441242.999, 100, 0.9, 0.001),
 (4980602.221, 500, 1, 0.1),
 (4909157.66, 500, 0.9, 0.1),
 (4859228.28, 500, 0.8, 0.1),
 (4802858.049, 200, 1, 0.1),
 (4795838.522, 200, 0.9, 0.1),
 (4760270.397, 200, 0.8, 0.1),
 (4662747.643, 100, 0.9, 0.1),
 (4620992.946, 100, 1, 0.1),
 (4520889.74, 100, 0.8, 0.1)]

for i in range(len(tuning_data_no_elitism)):
    tuning_data_no_elitism[i] = (tuning_data_no_elitism[i][0],tuning_data_no_elitism[i][1], 
                              tuning_data_no_elitism[i][2], tuning_data_no_elitism[i][3], 'no_elit_no_migr')
    
print(tuning_data_no_elitism)

[(6148712.234999999, 500, 0.9, 0.01, 'no_elit_no_migr'), (6146679.5709999995, 500, 0.8, 0.017543859649122806, 'no_elit_no_migr'), (6146156.528, 500, 0.9, 0.017543859649122806, 'no_elit_no_migr'), (6146140.344, 500, 1, 0.01, 'no_elit_no_migr'), (6137345.623, 500, 1, 0.017543859649122806, 'no_elit_no_migr'), (6131581.3319999995, 500, 0.8, 0.01, 'no_elit_no_migr'), (6123178.155, 200, 0.8, 0.017543859649122806, 'no_elit_no_migr'), (6104209.779, 200, 1, 0.01, 'no_elit_no_migr'), (6099314.348, 200, 0.9, 0.017543859649122806, 'no_elit_no_migr'), (6087498.12, 200, 1, 0.017543859649122806, 'no_elit_no_migr'), (6073420.446, 100, 0.9, 0.017543859649122806, 'no_elit_no_migr'), (6067200.394, 100, 0.8, 0.017543859649122806, 'no_elit_no_migr'), (6060090.344, 200, 0.8, 0.01, 'no_elit_no_migr'), (6056372.988, 100, 1, 0.017543859649122806, 'no_elit_no_migr'), (6031386.987, 200, 0.9, 0.01, 'no_elit_no_migr'), (6031222.557, 500, 1, 0.001, 'no_elit_no_migr'), (6024036.263, 100, 1, 0.01, 'no_elit_no_migr'),

In [11]:
tuning_data_elitism_migration = [(6151036.403, 500, 1, 0.017543859649122806),
 (6148284.494, 500, 0.9, 0.017543859649122806),
 (6144883.415999999, 500, 0.8, 0.01),
 (6144711.563999999, 500, 1, 0.01),
 (6135703.629, 500, 0.8, 0.017543859649122806),
 (6125407.726, 200, 0.9, 0.017543859649122806),
 (6121445.506, 100, 0.9, 0.017543859649122806),
 (6120201.197, 500, 0.9, 0.01),
 (6110270.227, 200, 1, 0.017543859649122806),
 (6105125.484, 200, 0.8, 0.017543859649122806),
 (6096314.24, 200, 0.9, 0.01),
 (6084734.848, 100, 1, 0.017543859649122806),
 (6081133.353, 200, 1, 0.01),
 (6071861.793, 200, 0.8, 0.01),
 (6065600.717, 500, 1, 0.001),
 (6054539.314, 100, 1, 0.01),
 (6052404.814, 500, 0.9, 0.001),
 (6048056.814, 100, 0.8, 0.017543859649122806),
 (6017255.733, 100, 0.8, 0.01),
 (6010828.816, 100, 0.9, 0.01),
 (5996285.937, 500, 0.8, 0.001),
 (5842992.513, 200, 1, 0.001),
 (5836531.457, 200, 0.9, 0.001),
 (5824727.717, 200, 0.8, 0.001),
 (5715033.272, 100, 1, 0.001),
 (5709434.537, 100, 0.8, 0.001),
 (5528802.921, 100, 0.9, 0.001),
 (5079972.3379999995, 500, 0.9, 0.1),
 (5002283.778, 500, 1, 0.1),
 (4983574.278, 500, 0.8, 0.1),
 (4949339.091, 200, 0.8, 0.1),
 (4932930.139, 200, 0.9, 0.1),
 (4916070.34, 200, 1, 0.1),
 (4877656.171, 100, 1, 0.1),
 (4855472.8719999995, 100, 0.8, 0.1),
 (4772316.27, 100, 0.9, 0.1)]

for i in range(len(tuning_data_elitism_migration)):
    tuning_data_elitism_migration[i] = (tuning_data_elitism_migration[i][0],tuning_data_elitism_migration[i][1], 
                              tuning_data_elitism_migration[i][2], tuning_data_elitism_migration[i][3], 'elit_and_migr')
    
print(tuning_data_elitism_migration)

[(6151036.403, 500, 1, 0.017543859649122806, 'elit_and_migr'), (6148284.494, 500, 0.9, 0.017543859649122806, 'elit_and_migr'), (6144883.415999999, 500, 0.8, 0.01, 'elit_and_migr'), (6144711.563999999, 500, 1, 0.01, 'elit_and_migr'), (6135703.629, 500, 0.8, 0.017543859649122806, 'elit_and_migr'), (6125407.726, 200, 0.9, 0.017543859649122806, 'elit_and_migr'), (6121445.506, 100, 0.9, 0.017543859649122806, 'elit_and_migr'), (6120201.197, 500, 0.9, 0.01, 'elit_and_migr'), (6110270.227, 200, 1, 0.017543859649122806, 'elit_and_migr'), (6105125.484, 200, 0.8, 0.017543859649122806, 'elit_and_migr'), (6096314.24, 200, 0.9, 0.01, 'elit_and_migr'), (6084734.848, 100, 1, 0.017543859649122806, 'elit_and_migr'), (6081133.353, 200, 1, 0.01, 'elit_and_migr'), (6071861.793, 200, 0.8, 0.01, 'elit_and_migr'), (6065600.717, 500, 1, 0.001, 'elit_and_migr'), (6054539.314, 100, 1, 0.01, 'elit_and_migr'), (6052404.814, 500, 0.9, 0.001, 'elit_and_migr'), (6048056.814, 100, 0.8, 0.017543859649122806, 'elit_and_

In [12]:
tuning_data_no_elitism_migration = [(6148971.013, 500, 1, 0.01),
 (6148087.92, 500, 1, 0.017543859649122806),
 (6146698.551, 500, 0.9, 0.017543859649122806),
 (6143838.6049999995, 500, 0.8, 0.017543859649122806),
 (6140194.619, 500, 0.8, 0.01),
 (6129599.678, 200, 1, 0.017543859649122806),
 (6128092.166, 200, 0.9, 0.017543859649122806),
 (6115455.506, 500, 0.9, 0.01),
 (6106649.362, 200, 0.8, 0.017543859649122806),
 (6097232.341, 200, 0.8, 0.01),
 (6095438.024, 500, 0.9, 0.001),
 (6068637.379, 200, 0.9, 0.01),
 (6066762.222, 200, 1, 0.01),
 (6050373.821, 100, 1, 0.017543859649122806),
 (6050246.996, 500, 1, 0.001),
 (6047130.399, 100, 0.8, 0.01),
 (6046591.481, 100, 0.8, 0.017543859649122806),
 (6041961.114, 100, 0.9, 0.017543859649122806),
 (6040103.001, 500, 0.8, 0.001),
 (5997696.447, 100, 0.9, 0.01),
 (5991088.158, 100, 1, 0.01),
 (5892165.871, 200, 1, 0.001),
 (5852971.688, 200, 0.9, 0.001),
 (5833457.4969999995, 200, 0.8, 0.001),
 (5713666.824, 100, 1, 0.001),
 (5667653.621, 100, 0.9, 0.001),
 (5416481.652, 100, 0.8, 0.001),
 (4973407.168, 500, 1, 0.1),
 (4848568.097, 500, 0.8, 0.1),
 (4822951.754, 500, 0.9, 0.1),
 (4760286.584, 200, 0.9, 0.1),
 (4748095.851, 200, 1, 0.1),
 (4698423.278, 100, 0.8, 0.1),
 (4665172.188, 100, 1, 0.1),
 (4636472.434, 200, 0.8, 0.1),
 (4601687.935, 100, 0.9, 0.1)]

for i in range(len(tuning_data_no_elitism_migration)):
    tuning_data_no_elitism_migration[i] = (tuning_data_no_elitism_migration[i][0],
                                           tuning_data_no_elitism_migration[i][1], 
                                           tuning_data_no_elitism_migration[i][2], 
                                           tuning_data_no_elitism_migration[i][3], 'no_elit_migr')
    
print(tuning_data_no_elitism_migration)

[(6148971.013, 500, 1, 0.01, 'no_elit_migr'), (6148087.92, 500, 1, 0.017543859649122806, 'no_elit_migr'), (6146698.551, 500, 0.9, 0.017543859649122806, 'no_elit_migr'), (6143838.6049999995, 500, 0.8, 0.017543859649122806, 'no_elit_migr'), (6140194.619, 500, 0.8, 0.01, 'no_elit_migr'), (6129599.678, 200, 1, 0.017543859649122806, 'no_elit_migr'), (6128092.166, 200, 0.9, 0.017543859649122806, 'no_elit_migr'), (6115455.506, 500, 0.9, 0.01, 'no_elit_migr'), (6106649.362, 200, 0.8, 0.017543859649122806, 'no_elit_migr'), (6097232.341, 200, 0.8, 0.01, 'no_elit_migr'), (6095438.024, 500, 0.9, 0.001, 'no_elit_migr'), (6068637.379, 200, 0.9, 0.01, 'no_elit_migr'), (6066762.222, 200, 1, 0.01, 'no_elit_migr'), (6050373.821, 100, 1, 0.017543859649122806, 'no_elit_migr'), (6050246.996, 500, 1, 0.001, 'no_elit_migr'), (6047130.399, 100, 0.8, 0.01, 'no_elit_migr'), (6046591.481, 100, 0.8, 0.017543859649122806, 'no_elit_migr'), (6041961.114, 100, 0.9, 0.017543859649122806, 'no_elit_migr'), (6040103.001,

In [58]:
average_time = []
for i in range(1000):
    test = np.random.randint(0, 4, 57).tolist()
    average_time.append(production(test, 'scale_total', 'half year')[4])
    
mean(average_time)

0.003968819379806519

In [60]:
average_time = []
for i in range(1000):
    average_time.append(production(test, 'scale_total', 'half year')[4])
    
mean(average_time)

0.0009481730461120605

In [62]:
0.003968819379806519 / 0.0009481730461120605

4.18575427352684

In [63]:
genetic_algorithm(57,500, 600, 0.8, 'rush', 'half year', 1 / 57, True, False)

Time taken: 219.4539315700531 seconds


[[0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2],
 (796905.0566316111, 161.0, 38.0, 0.013004541397094727)]

In [64]:
genetic_algorithm(57,500, 600, 0.8, 'scale_total', 'half year', 1 / 57, True, False)

Time taken: 217.69661140441895 seconds


[[1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 (6158048.018822445, 191.0, 118.0, 657035.8749602238, 0.008969783782958984)]

In [65]:
genetic_algorithm(57,500, 600, 0.8, 'scale_civs', 'half year', 1 / 57, True, False)

Time taken: 294.61813020706177 seconds


[[1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  0,
  0,
  0,
  1,
  1,
  1,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 (2543185.812488124, 167.0, 157.0, 591814.9840359812, 0.010972976684570312)]